# Colabs Online Processing for HABSpec 
as of 2021, August 20

C. W. Wright Lidar532@gmail.com

# Run these cells first to get started.

In [ ]:
#@title Copy and untar to 2021-0717 NO-PIX data from the Gdrive to /content { form-width: "30%" }
if __name__ == '__main__':
  ! cp /content/drive/MyDrive/Missions/2021-0717-HAB-pie-NO-PIX.tar.gz /content/
  ! tar -xzf /content/drive/MyDrive/Missions/2021-0717-HAB-pie-NO-PIX.tar.gz
  print('Operation completed.')

# Load required local functions.






### HyperSpec class & methods

#### Docs

##### methods and Data in HyperSpectral Class:

Data:
* Fraunhofer_lines
* HG_lines
* Xpixels
* wavelengths

Functions:
* def configure_json_spectra(f)
* def calibrate_using_2_wavelengths(self, pixel0=0, wavelength0=0, pixel1=0, wavelength1=0 )
* def read_spectra(self, f, remove_bias = True, y_average=True )
* def hhmmss_to_sod(self, hhmmss, Usecs=0)
* def nearest_rgb_image(self, fn)
* 

#### Code

In [ ]:
#@title Class: Hyperspectral. { form-width: "30%" }
import glob
import numpy as np
import pandas as pd
import cv2 as cv
import datetime as dt
from geographiclib.geodesic import Geodesic
import glob
from google.colab.patches import cv2_imshow # for image display
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import pandas as pd
from PIL import Image   
from skimage import io
import time

class HyperSpec:
  asof_str = 'HyperSpec 2021-0823-1931'

  Fraunhofer_lines = {
                    'A' :[ 'O2', 759.370  ],
                    'B' :[ 'O2', 686.719  ], 
                    'C' :[ 'Ha', 656.281  ],
                    'a' :[ 'O2', 627.661  ],
                    'D1':[ 'Na', 589.592  ],
                    'D2':[ 'Na', 588.995  ],
                    'D3':[ 'He', 587.5618 ],
                  'e-hg':[ 'Hg', 546.073  ],
                    'E2':[ 'Fe', 527.039  ],
                    'b1':[ 'Mg', 518.362  ],
                    'b2':[ 'Mg', 517.270  ],
                    'b3':[ 'Mg', 516.891  ],
                    'b4':[ 'Mg', 516.733  ],
                    'c' :[ 'Fe', 495.761  ],                                                            
                    'F' :[ 'Hb', 486.134  ],
                    'd' :[ 'Fe', 466.814  ],
                  'e-Fe':[ 'Fe', 438.355  ],
                    'G' :[ 'Fe', 430.790  ],
                    'G2':[ 'Ca', 430.774  ],
                    'H' :[ 'Ca', 396.847  ] 
  }

  # Re HG_Lines.  Wavelengths taken from the following sources.
  # See: http://hyperphysics.phy-astr.gsu.edu/hbase/quantum/atspect2.html
  # and: https://commons.wikimedia.org/wiki/File:Fluorescent_lighting_spectrum_peaks_labelled.gif
  # and: https://en.wikipedia.org/wiki/Fluorescent_lamp#/media/File:Spectra-Philips_32T8_natural_sunshine_fluorescent_light.svg
  HG_lines = {  
              'Hg-404' :[ 'Hg', 404.6563 ],
              'Hg-436' :[ 'Hg', 435.8328 ],
              'Hg-543' :[ 'Hg', 543.6    ],
              'Hg-546' :[ 'Hg', 546.0735 ],
              'Hg-576' :[ 'Hg', 576.959  ],
              'Hg-579' :[ 'Hg', 579.065  ],
              'Hg-611' :[ 'Hg', 610.8    ],
              'Hg-615' :[ 'Hg', 614.9475 ]
      
  }

  def __init__(self):
    self.Xpixels = 0
    self.summed_rows  = 0
    self.Xpixels     = np.array(1)
    self.wavelengths = np.array(1)
    self.create_time = dt.datetime.utcnow()

  def version(self):
      return f'{self.create_time} HyperSpec Class instantiated. asof: {self.asof_str}'

  def configure_json_spectra(self,f):
    '''
    Configures HabSpec internal setting according to settings within json file 'f'.

    Parameters:
    -----------
    f : str
        HabSpec json spectra full path and file name.

    Returns:
    --------
        None.
    '''
    spec = pd.read_json(f)                      # get a spectra fron a json file
    y = np.array(spec['hab_spec'].spectra )     # Load the y values to an np array
    self.summed_rows = spec['hab_spec'].summed_rows
    self.Xpixels = np.arange(0,y.size )         # Construct an array of X values
    self.wavelengths = np.arange( y.size )
    return f

  def calibrate_using_2_wavelengths(self, pixel0=0, wavelength0=0, pixel1=0, wavelength1=0 ):
    '''
    Generate a Numpy array of calibration wavelenghts for each pixel. configure_json_spectra(f)
    must be called beforehand inrder to set the correct number of pixels.

    Parameters:
    -----------
    pixel0 : int, default 0
    wavelength0 : float, default 0
    pixel1 : int, default 0
    wavelength1 : float, default 0

    Returns:
    --------
    Float array of calibrated wavelengths for each pixel.  The returned array is the
    same size as self.Xpixels.

    Example:
    ---------
    # Configure the spectra by reading a spectra file.
    fn = '/content/drive/MyDrive/Missions/2021-0717-HAB-pie/165347/hab_spectra/2021-0717-165348-272814-spec.json'
    hs.configure_json_spectra(fn)

    # Calibrate the spectra.  Pixel 73 is at 430.774nm, and pixel 941 is at 759.370nm
    hs.calibrate_using_2_wavelengths(pixel0=73, wavelength0=430.774, pixel1=941, wavelength1=759.370 )

    References:
    -----------
    The following are wavelength calibration sources.
    See: http://hyperphysics.phy-astr.gsu.edu/hbase/quantum/atspect2.html
    and: https://commons.wikimedia.org/wiki/File:Fluorescent_lighting_spectrum_peaks_labelled.gif
    and: https://en.wikipedia.org/wiki/Fluorescent_lamp#/media/File:Spectra-Philips_32T8_natural_sunshine_fluorescent_light.svg
    '''
    self.dp = pixel1      - pixel0              # pixel delta
    self.dw = wavelength1 - wavelength0         # Wavelength delta
    self.slope = self.dw/self.dp                # Linear slope
    self.wavelengths = wavelength0 + (self.Xpixels-pixel0) * self.slope
    return self.wavelengths


  def read_spectra(self, f, remove_bias = True, y_average=True ): 
    """
    Reads a Json hyperspectra file.

    Parameters:
    -----------
    f : str
      The full path name of a hyperspectra Json file.
    remove_bias : Default True
      Subtracts the minimum y value from the array of y values to remove dark current. 
      The minimum value will typically be found in the IR side and from the pixel 
      sensors that are optically obscured.
    y_average : Default True
      The Json values are the sum total of each pixel column. Each column contains 800
      pixels. Setting this to True causes the read y values to be divided by 800.

    Returns:
    --------
    Numpy array
      An array of numpy float values representing the intensity values at each pixel. 

    Description:
    ------------
    Reads a Json spectra from a file.

    Examples:
    ---------
    fn = '/content/drive/MyDrive/Missions/2021-0717-HAB-pie/165347/hab_spectra/2021-0717-165348-272814-spec.json'
    s = hs.read_spectra(fn)
    """
    spec = pd.read_json(f)
    if self.summed_rows == 0 :
      self.configure_json_spectra(f)
    y = np.array(spec['hab_spec'].spectra)

    if y_average == True:
      y = y / self.summed_rows

    if remove_bias == True:
      y = y - y.min()
    return y


  '''
    Typical filename.
      '/content/drive/MyDrive/Missions/2021-0717-HAB-pie/141625/hab_spectra/2021-0717-141658-955482-spec.json'

  '''

  def hhmmss_to_sod(self, hhmmss, Usecs=0):
    """
    Converts a time string in 'HHMMSS' format to a seconds-of-the-day
    including an optional microseconds fraction.

    Parameters:
    ----------- 
    hhmmss : str
    Usecs  : str, default=0

    Returns: 
    --------
    float
      Seconds of the day including the fractional part computed from `Usecs`
    
    Description:
    ------------
    hhmmss where hh is hours, mm is minutes, and ss is seconds. Example '123456'
    is 12 hours, 34 minutes, and 56 seconds. A string representing the number of
    microseconds since the last second increment, ie the fractional part of a second. 
    Example: '954561' represents 0.954561 seconds, or 954,561 microseconds since the
    last seconds rollover. Returns A single floating point value of the seconds since
    midnight plus the fractional seconds.

    Examples:
    ---------
    To be added.

    """
    hh = int(hhmmss[0:2])
    mm = int(hhmmss[2:4])
    ss = int(hhmmss[4:6])
    fsod = hh*3600 + mm*60 + ss + float(Usecs) * 1e-6
    return fsod

  #debug_nearest_rgb_image = False
  def nearest_rgb_image(self, fn, debug=False):
    """
    Returns the path/filename to the RGB photo closes in time to `fn`.  `fn` is the
    filename of a Json hyperspectral file.

    Parameters:
    -----------
    fn : str
      Full path/filename of a hyperspectral Json file.

    Returns:
    --------
    str
      A string path/filename of the closes RGB photo.

    Examples:
    ---------

    References:
    -----------
    """
    if debug:
      print(f'debug_nearest_rgb_image(fn): fn={fn}')
    fn_parts = fn.split('/')   # Split path by '/'
    if debug:
      print(f'debug_nearest_rgb_image(fn): fn_parts={fn_parts}')
    rgb_p = fn_parts                          # Make a copy to build the rgb path/filename in.
    if debug:
      print(f'debug_nearest_rgb_image(fn): rgb_p={rgb_p}')    
    rgb_p[-2] = 'hab_rgb'                     # Change the subdir to point to hab_rgb
    rgb_fn = rgb_p[-1].split('-')             # Split the filename by '-' to access parts.
    rgb_fn[-2] = '*'                          # Replace the microseconds with '*' wildcard.
    rgb_fn[-1] = 'rgb.jpg'                    # Change the file tail to rgb.jpg
    rgb_p[-1] = '-'.join(rgb_fn)              # Glue the name back together
    rgb_p2 = '/'.join(rgb_p)                  # Now glue the whole path back together
    rv = glob.glob(rgb_p2)[0]                 # Return the first entry incase there are more than 1.
    if debug:
      print(f'debug_nearest_rgb_image(fn): rv={rv}') 
    return rv

  #=================================================================================
  def get_list_of_json_spectra(self, p):
    ''' Returns a list of Json spectra full path filenames found in subdirs under "p". '''
    gstr = f'{p}/*/*/*-spec.json'
    lst = glob.glob(gstr, recursive=True)
    return lst

  #=================================================================================
  def extract_hms(self, fn):
    '''Extract the HHMMSS and Fsecs strings from filename "fn"

    Returns:
    --------
    ( hhmmss:str, fsecs:str, hhmmss.fsecs:float, sod.fsecs:float )
    '''
    lst = fn.split('-')
    sod = self.hhmmss_to_sod( lst[-3], Usecs=lst[-2] )
    return ( lst[-3], lst[-2], float(lst[-3])+float(lst[-2])*1e-6, sod  )

  #=================================================================================
  def json_specs_to_df(self, specs ):
    '''
    Extracts the hhmmss and fsecs from 'specs' filename, converts
    the hhmmss and fsecs strings to float SOD.fsecs.

    Parameters:
    specs: list  A list of json spectra files.

    Returns:
    --------
    A DataFrame of sod.fsecs:float, hhmmss:str, Json_spec:str
    '''
    # Extract the hhmmss and fsecs from each file name,
    # convert the hhmmss and fsecs to float SOD.fsecs, 
    # and create a list of each.
    hhmmss = []
    fsecs  = []
    sod    = []
    for v in specs:
      tx = self.extract_hms( v )
      hhmmss.append( tx[0] )   # Build the list if hhmmss strings.
      sod.append( tx[3] )      # Build the list of sod floats.

    # Now convert the  lists to a Pandas dataframe.
    dct = {'sod':sod, 'hhmmss':hhmmss, 'Json_spec':specs}
    df = pd.DataFrame( dct )
    df.sort_values(by=['sod'], inplace=True)              # Sort the data in time order.
    return df

  #=================================================================================
  #debug_read_gps_file_to_df = False
  def read_gps_file_to_df(self, ifn, debug=False):
    '''
    Read a GPS datafile into a dataframe and convert the HH:MM:SS
    to add an SOD column.

    Parameters:
    -----------
    ifn : str
        Input GPS datafile full path name

    Returns:
    --------
    Pandas Dataframe of the GPS data.
    '''
    gps_df = pd.read_csv(ifn, sep='\s+', comment='#')
    gps_df.sort_values(by=['HMS'], inplace=True)
    # Convert the ASCII HH:MM:SS from the GPS to seconds of the day (SOD) and
    # add an 'SOD' column to the gps dataframe.
    sod_lst = []
    for t in gps_df['HMS']:
      sod_lst.append( self.hms2sod(t))
    gps_df['SOD'] = sod_lst

    # Compute, and add 'Course' to dataframe
    course = []
    for i in  range(len(gps_df)-1):
      course.append ( self.get_bearing( 
          lat1=gps_df['Lat'].iloc[i], long1=gps_df['Lon'].iloc[i], 
          lat2=gps_df['Lat'].iloc[i+1], long2=gps_df['Lon'].iloc[i+1] 
          ) 
      )
    course.append( 0.0 )      # To make the same length
    gps_df['Course'] = course
    return gps_df

  #=================================================================================
  # See: https://numpy.org/doc/stable/reference/generated/numpy.interp.html
  #debug_compute_gps_positions = False
  def compute_gps_positions( self, gps, spec_df, debug=False):
    '''
    Interpolates spectra positions from gps.

    Parameters:
    -----------
    gps : dataframe
        A dataframe of overlapping gps position data.

    spec_df : dataframe
        A dataframe of values vs time from the spectrometer.

    Returns:
    --------
    See: https://numpy.org/doc/stable/reference/generated/numpy.interp.html
    '''
    spec_df['Lat']    = np.interp(spec_df['sod'], gps['SOD'], gps['Lat'])
    spec_df['Lon']    = np.interp(spec_df['sod'], gps['SOD'], gps['Lon'])
    spec_df['Elev']   = np.interp(spec_df['sod'], gps['SOD'], gps['Elev'])
    spec_df['Course'] = np.interp(spec_df['sod'], gps['SOD'], gps['Course'])
    return spec_df

  #@title Get bearing between two lat/lon pairs { form-width: "30%" }
  #=================================================================================
  #debug_get_bearing = False
  def get_bearing(self, lat1=0, lat2=0, long1=0, long2=0, debug=False):
    '''
    Comutes and returns the bearing between two lat/lon pairs.
    See:
        http://shorturl.at/atGHN
    '''
    brng = Geodesic.WGS84.Inverse(lat1, long1, lat2, long2)['azi1']
    return brng

  #=================================================================================
  #debug_hms2sod = False
  def hms2sod(self, str, debug=False):
    '''
    Converts an ASCII string in the form 'HH:MM"SS' to seconds of the day. Works with
    fractional seconds.
    '''
    if len(str) == 8:
      ts = time.strptime( str, '%H:%M:%S')
      fs = 0.0
    else:
      ts = time.strptime( str[0:8], '%H:%M:%S')
      fs = float(str[8:])
    sod = ts[3]*3600 + ts[4]*60 + ts[5] + fs
    return sod

  #=================================================================================
  #debug_w_range_pixels = True
  def w_range_pixels(self, x,a,b, debug=False):
    '''
    Returns a pixel index list of all pixels between wavelength 'a' and 'b'. 
    Parameters:
      x   A numpy array of wavlengths for each pixel
      a   Starting wavelength
      b   Ending wavelength
    
    Returns:
      A list of index values from wavelength 'a' to 'b'
    '''
    rv = np.where(np.logical_and( x>a, x<b) )
    return rv

  #=================================================================================
  #debug_get_fluorescence = False
  def get_fluorescence(self, x, y, fl_start=0, fl_stop=0, base_start=0, base_stop=0, debug=False ):
    '''
    '''
    fl_sig = y[self.w_range_pixels(x, fl_start, fl_stop )].mean()         # Get the Fluor signal mean value
    by = y[self.w_range_pixels(x, base_start, base_start+1)].mean()
    ey = y[self.w_range_pixels(x, base_stop, base_stop+1)].mean()
    cw = (fl_stop - fl_start) / 2  + fl_start                                       # Compute center wavelength
    dydx = (ey - by)/(base_stop - base_start )
    sf = dydx * (cw - base_start)
    rv = fl_sig + sf
    if debug:
      print('get_fluorescence():', fl_sig, by, ey, cw, dydx, sf, rv)
    return rv

  #=================================================================================
  def get_fluorescence_700(self, x, y):
    '''
    Returns the fluorescence value at 700nm.
    '''
    rv = self.get_fluorescence( x, y, fl_start=693, fl_stop=710, base_start=668, base_stop=740 )
    return rv

  #=================================================================================
  def get_fluorescence_683(self, x, y):
    '''
    Returns the fluorescence value at 683nm.  683nm is Chlorophyll
    '''
    rv = self.get_fluorescence( x, y, fl_start=678, fl_stop=688, base_start=668, base_stop=740 )
    return rv

  #=================================================================================
  def now_utc(self, fmt='%Y-%m%d %H:%M:%S'):
    """ Return the current UTC date and time as a string.  Example return: '2021-0812 15:39:41'.
    See: http://shorturl.at/koOQ7 for timeformat options & directives.
    """
    t = dt.datetime.utcnow()
    ts = f'{t:{fmt}}'
    return ts

  #=================================================================================
  #debug_is_water = False
  def is_water(self, x,y, debug=False):
    '''
    Returns the mean signal value between 840nm and 860nm wavelengths.  Since water absorbs IR.

    Inputs:
      x   A numpy array of wavlengths for each pixel
      y   A numpy array of intensity values at each wavelength.  x and y mus be the same size.

    Returns:
      The mean signal value between 840nm and 860nm.  The signal level is not currently normalized
      for anything, exposure, etc.  Threshold is around 4.0.  Above 4, land or glint.

    References: 
    Application of the water-related spectral reflectance indices: A review
    https://www.sciencedirect.com/science/article/abs/pii/S1470160X18308215

    '''
    rv = y[self.w_range_pixels(x,840,860)].mean()
    return rv

  #=================================================================================
  #debug_get_list_of_flight_lines = False
  def get_list_of_flight_lines(self, p, debug=False):
    '''Returns a list of flightline subdirs on path p.
    
    Parameters:
    -----------
    p : str
        Path name.

    Returns:
    --------
    list
        A list of full pathnames to individul flightlines.
    '''
    l = glob.glob(p+'/[0-9]*[!a-z]')
    return l

  #=================================================================================
  #debug_get_list_of_hab_files = False
  def get_list_of_hab_files(self, p, subdir='', ext='', debug=False):
    '''Returns a list of hab files in p/subdir with the specified file extension.  
    
    Parameters:
    p : str
    subdir : str Default = ''
    ext : str Default = ''

    Returns:
    --------
    list
        A list of  full pathnames.
    '''
    gs = p+f'/{subdir}/*.'+ext
    js = glob.glob(gs)
    return js

if __name__ == '__main__':
  ths = HyperSpec()
  print(f'{ths.now_utc()}: The HyperSpectral class has been created.')
  del ths


In [ ]:
#@title Test: HyperSpectral.{ form-width: "30%" }
#@title Test hs.read_gps_file_to_df(Gps_File_Name) { form-width: "30%" }
#TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT
# Test read_gps_file_to_df(Gps_File_Name)
debug_read_gps_file_to_df = False
if __name__ == '__main__':
  if debug_read_gps_file_to_df:
    import numpy as np
    import pandas as pd
    print('Testing: read_gps_file_to_df(Gps_File_Name)')
    if 'HyperSpec' in dir():
      hs = HyperSpec()
    Gps_File_Name = '/content/drive/MyDrive/Missions/2021-1208-HAB-Lake-Parker/2021-1208/2021-1208-175324-910850-mission-gps.txt'
    gps_df = hs.read_gps_file_to_df(Gps_File_Name, debug=False)
    display(gps_df)

#@title Testing hs.get_bearing() { form-width: "30%" }
#TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT
if __name__ == '__main__':
  debug_get_bearing = False
  if debug_get_bearing:
    lst = [ [30,29, -75, -75],
            [29,30, -75, -75],
            [30,30, -76, -75],
            [30,30, -75, -76] ]
    print('\nTesting get_bearing()')
    for t in lst:
      v = hs.get_bearing( lat1=t[0], lat2=t[1], long1=t[2], long2=t[3] )
      print( f'  {t}, {v:6.2f}')

#@title Testing: hs.hms2sod( t ) { form-width: "30%" }
#TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT
if __name__ == '__main__': 
  debug_hms2sod = False
  if debug_hms2sod:
    print('\nTesting: hs.hms2sod( t )')
    tt = ['00:00:00', '0:0:0', '23:59:59', '12:00:00']
    for t in tt:
      rv = hs.hms2sod( t )
      print(t, rv)

#@title Testing:hs.compute_gps_positions(gps_df, df)') { form-width: "30%" }
#TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT
if __name__ == '__main__': 
  debug_compute_gps_positions = False
  if debug_compute_gps_positions:
    print('\nTesting: hs.compute_gps_positions(gps_df, df)')
    gps_df = hs.read_gps_file_to_df(Gps_File_Name)
    sdf = hs.compute_gps_positions(gps_df, df)
    display(sdf)

#TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT
if __name__ == "__main__": 
  debug_w_range_pixels = False
  if debug_w_range_pixels:
    lst = hs.w_range_pixels( x, 678, 680)
    print(f'\nhs.w_range_pixels(): Pixel index values between 678 and 680nm: {lst}\n y values:{ y[lst]}')

#TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT
if __name__ == "__main__":
  debug_get_fluorescence = False
  if debug_get_fluorescence:
    print('\nTesting: hs.get_fluorescence_683(x, y )')
    rv = hs.get_fluorescence_683(x, y )
    rv2 = hs.get_fluorescence_700(x, y )
    print(f'  get_fluorescence_683():   returned: {rv}\n'
          f'  get_fluorescence_700:     returned: {rv2}')
  
#TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT
if __name__ == "__main__":
  debug_is_water = False
  if debug_is_water == True:
    print('\nTesting: hs.is_water(x,y)')
    print( f'  test hs.is_water(): { hs.is_water(x,y) }' )

#TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT
if __name__ == "__main__":
  debug_get_list_of_flight_lines = False
  if debug_get_list_of_flight_lines:
    print('\nTesting: hs.get_list_of_flight_lines(p)')
    test_path_list = get_list_of_flight_lines('/content/2021-0717-HAB-pie-NO-PIX')
    display(test_path_list)

#TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT
debug_get_list_of_hab_files = False
if __name__ == "__main__":
  if debug_get_list_of_hab_files:
    print('\nTesting: hs.get_list_of_hab_files()')
    flight_line_list = hs.get_list_of_flight_lines('/content/drive/MyDrive/Missions/2021-0717-HAB-pie')
    rv_get_list_of_hab_files = hs.get_list_of_hab_files( flight_line_list[1], subdir='hab_spectra', ext='json')
    display(rv_get_list_of_hab_files)

#@title Tests: HyperSpectral Wavelength Calibration. { form-width: "30%" }
#TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT
# Create a spectra obj

if __name__ == '__main__':
  debug_hyperspectral = False
  if debug_hyperspectral:
    import datetime
    print(f'{datetime.datetime.utcnow() }: Testing creating HyperSpectral')
    # Make sure the 'HyperSpec' class cell has been run, and if so, use it.
    if 'HyperSpec' in dir():
      hs = HyperSpec()

      # Use a json spectra to initialize things.
      print(f'{hs.now_utc()}: Loading json file to calibrate wavelength.')
      fn = '/content/drive/MyDrive/Missions/2021-0717-HAB-pie/165347/hab_spectra/2021-0717-165348-272814-spec.json'
      hs.configure_json_spectra(fn)

      # Calibrate the spectra for wavelength in nanometers
      print(f'{hs.now_utc()}: Calibrating wavelength.')
      hs.calibrate_using_2_wavelengths(pixel0=73, wavelength0=430.774, pixel1=941, wavelength1=759.370 )

      print('\nWavelength Calibration Data:\n'
      '                     First      Second\n'
      f'   Wavelengths (nm): {hs.wavelengths[73]:7.3f}      {hs.wavelengths[941]:7.3f}\n'
      f'     Pixel Location: {73:3d}          {941:3d}\n')

      # Read a spectra array from a file into s
      print(f'{hs.now_utc()}: Reading a spectra into "s"')
      s = hs.read_spectra( fn )

#@title Tests: hs.nearest_rgb_image( Spectra_File_Name ) { form-width: "30%" }
#TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT
if __name__ == '__main__':
  debug_show_RGB_at_spec = False
  if not 'HyperSpec' in dir():
    print(f"{hs.now_utc()}: The HyperSpec class isn't loaded.  Load the class, and rerun.")
  else:
    hs = HyperSpec()
    # import glob
    #img_path = '/content/drive/MyDrive/Missions/2021-0717-HAB-pie/165347/hab_rgb/2021-0717-170033-323503-rgb.jpg'
    #print(Spectra_File_Name,"\n",img_path)
    if debug_show_RGB_at_spec:
      Spectra_File_Name = '/content/drive/MyDrive/Missions/2021-0717-HAB-pie/164443/hab_spectra/2021-0717-164525-775042-spec.json'
      print(
      f'    Input Json spectra file: {Spectra_File_Name}\n'
      f'        Nearest RGB File is: {hs.nearest_rgb_image( Spectra_File_Name )}'  )

#@title Tests: hs.nearest_rgb_image( Spectra_File_Name ) { form-width: "30%" }
#TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT
if __name__ == '__main__':
  print(f'{hs.now_utc()}: HyperSpectral Tests have been executed.')


### Local required methods.

In [ ]:
#@title w_min( x, y, wstart = 0, wstop = 0, debug=False) { form-width: "30%" }
#=================================================================================
#debug_get_fluorescence = False
def  w_min( x, y, w_start = 0, w_stop = 0, debug=False):
  '''
  '''

  sig = y[hs.w_range_pixels(x, w_start, w_stop )].min()         # 
  rv = sig
  if debug:
    print('w_min():', x, y, wstart, wstop, sig )
  return rv

#@title w_max( x, y, wstart = 0, wstop = 0, debug=False)
#=================================================================================
#debug_get_fluorescence = False
def  w_max( x, y, w_start = 0, w_stop = 0, debug=False):
  '''
  '''
  sig = y[hs.w_range_pixels(x, w_start, w_stop )].max()         #
  rv = sig
  if debug:
    print('w_max():', x, y, wstart, wstop, sig )
  return rv

if __name__ == '__main__':
  from matplotlib import pyplot as plt
  if not 'HyperSpec' in dir():
    print(f"{hs.now_utc()}: The HyperSpec class isn't loaded.  Load the class, and rerun.")
  else:
    hs = HyperSpec()
    fn = '/content/drive/MyDrive/Missions/2021-1208-HAB-Lake-Parker/2021-1208/183807/hab_spectra/2021-1208-184034-847363-spec.json'
    spectra = hs.read_spectra( fn )
    print('testing w_min & w_max')
    wavelengths = hs.calibrate_using_2_wavelengths(         # Calibrate the spectra for wavelength in nanometers
        pixel0=73, wavelength0=430.774, 
        pixel1=941, wavelength1=759.370 
        )
    mn = w_min(hs.wavelengths, spectra, w_start=650.0, w_stop=700.0)
    mx = w_max( hs.wavelengths, spectra, w_start=673.0, w_stop=760.0)
    print( mn,
          mx)
    plt.plot(hs.wavelengths, spectra)
    plt.plot([650, 700],[mn,mn] )
    plt.plot([673, 760], [mx,mx])




In [ ]:
hs.summed_rows


In [ ]:
spectra

In [ ]:
#@title Load required local functions. { form-width: "30%" }
'''
'''
import cv2 as cv
import datetime as dt
from geographiclib.geodesic import Geodesic
import glob
from google.colab.patches import cv2_imshow # for image display
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import pandas as pd
from PIL import Image   
from skimage import io
import time
import datetime




#=================================================================================
#debug_chl_fl = False
#def chl_fl(x,y):
#  '''
#  Returns the value determined by the mean of 680-687nm / 670-678nm.  #
#
#  Inputs:
#     x   A numpy array of wavlengths for each pixel
#     y   A numpy array of intensity values at each wavelength.  x and y mus be the same size.#
#
#  Returns:
#    The mean signal value between 840nm and 860nm.  The signal level is not currently normalized
#    for anything, exposure, etc.
#  '''
#  rv = y[hs.w_range_pixels(x,680,687)].mean() / y[hs.w_range_pixels(x,670,678)].mean()
#  return rv

#TTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTTT
#if __name__ == "__main__" and debug_chl_fl==True:
#  print(f'\nTest chl_fl(): returned value: {chl_fl(x,y)}')

#=================================================================================
#@title Show the nearest image to a spectra file { form-width: "30%" }
debug_show_RGB_at_spec = True
def show_RGB_at_spec( fn ):
  """
    Locates and displays a photo taken near the time embedded in the Json file 'fn'. 

    Parameters:
    -----------
    fn : string, json spectra full path name

    Returns:
    --------
    None

    Description:
    ------------
    Extracts the time from the Json filename, and constructs a cooresponding filename
    for a jpg photo that is closeby.  It does not consider the fractional seconds.
    """
  if debug_show_RGB_at_spec == False: 
    print(f'show_RGB_at_spec(fn): fn={fn}')
  img_fig = figure(plot_width=400,
                   plot_height=400, 
                   x_range=(0,1), 
                   y_range=(0,1)
                  )
  img_path = hs.nearest_rgb_image( fn )
  print('Image file:',img_path)
  image = cv.imread(img_path)
  scale_percent = 25 # percent of original size
  width = int(image.shape[1] * scale_percent / 100)
  height = int(image.shape[0] * scale_percent / 100)
  dim = (width, height)
  resized = cv.resize(image, dim, interpolation = cv.INTER_AREA)
  cv2_imshow( resized )


if __name__ == '__main__':
  print(f'{datetime.datetime.utcnow()}: local functions loaded and ready.')


# Process data

In [ ]:
#@title Generate fluorescence values vs time and add in GPS positions and course. { form-width: "40%" }
import pandas as pd
if __name__ == '__main__':
  hs = HyperSpec()

  root_data_path = '/content/drive/MyDrive/Missions/2021-0717-HAB-pie/'
  Gps_File_Name = "/content/drive/MyDrive/Missions/2021-1208-HAB-Lake-Parker/2021-1208/all-gps.txt" #@param {type:"string"}
  Root_Data_Path = "/content/drive/MyDrive/Missions/2021-1208-HAB-Lake-Parker/2021-1208" #@param {type:"string"}


  all_specs = hs.get_list_of_json_spectra(Root_Data_Path) # Get a list of json spectra files.
  hs.configure_json_spectra(all_specs[0])                 # Configure spectra.

  wavelengths = hs.calibrate_using_2_wavelengths(         # Calibrate the spectra for wavelength in nanometers
      pixel0=73, wavelength0=430.774, 
      pixel1=941, wavelength1=759.370 
      )

  spec_df = hs.json_specs_to_df( all_specs )              # Convert to a dataframe.
  gps_df  = hs.read_gps_file_to_df(Gps_File_Name)         # Gather the GPS data to a dataframe.
  spec_df = hs.compute_gps_positions(gps_df, spec_df )    # Compute positions of each spectra.

  chl_fl_lst = []                                         # List to hold chlorophyll fluorescence
  fl700_lst  = []                                         # List to hold 700nm fluorescence
  print(f'{hs.now_utc()}: Computing chl Fl. & Fl-700') 
  print('Processing:')    
  i = 0
  for idx, row in spec_df.iterrows():
    i = i +  1
    if (i % 1000) == 0:
      print(f'   {i} of {len(spec_df)}')
    with open( spec_df['Json_spec'][idx], 'r') as sf:
      spectra = hs.read_spectra( sf )
      if hs.is_water(wavelengths,spectra  ) < 4.0:
        chl_fl_lst.append( hs.get_fluorescence_683(wavelengths, spectra ))
        fl700_lst.append(  hs.get_fluorescence_700(wavelengths, spectra ))
      else:
        chl_fl_lst.append(np.nan)
        fl700_lst.append(np.nan)

  spec_df['Chl_Fl'] = chl_fl_lst                             # Append the Chl_Fl to the dataframe.
  spec_df['fl_700'] = fl700_lst
  print(f'   {i} of {len(spec_df)}')
  ofn = hs.now_utc(fmt='/content/%Y-%m%d-%H%M%S-HABSpec-results.txt')
  spec_df.to_csv(ofn, float_format='%.6f', index=False)
  print('\nProcessed data are in the "spec_df" dataframe.')
  print(f'Results written to: {ofn}')
  print(f'{hs.now_utc()}: Completed.')


# Plots, Graphs, Maps 

## GPS Plots, Maps

In [ ]:
#@title Show just the GPS Track on a map { form-width: "30%" }
if __name__ == '__main__':
  Gps_File_Name = "/content/drive/MyDrive/Missions/2021-1208-HAB-Lake-Parker/2021-1208/all-gps.txt" #@param {type:"string"}
  mp = "ESRI Imagery" #@param ["ESRI Imagery", "Open Street Maps", "WorldMap" ]
  Track_Color = "white" #@param ["red", "green", "blue", "black", "white", "gray", "lightgray", "darkgray"]
  Point_Size = 2.0 #@param [".1", ".25", ".5", ".75", "1.0", "1.5", "2.0", "3.0", "5.0", "10", "20"] {type:"raw"}
 

  import pandas as pd
  from pathlib import Path

  if not Path('/usr/local/bin/pyproj').is_file():
    !pip install pyproj
  import bokeh.io
  bokeh.io.output_notebook()
  import bokeh.plotting
  from bokeh.plotting import figure
  from bokeh.io import output_notebook, show
  from bokeh.tile_providers import get_provider, WIKIMEDIA, CARTODBPOSITRON, STAMEN_TERRAIN, STAMEN_TONER, ESRI_IMAGERY, OSM
  from pyproj import Proj, transform

  import warnings
  warnings.filterwarnings("ignore")

  TOOLS = "pan,wheel_zoom,box_zoom,reset, save, undo, redo, hover"
  TOOLTIPS = [
      ("index", "$index")  #,
      #("(x,y)", "($x, $y)"),
      #("desc", "@desc"),
  ]

  # Create a spectra obj. Functions to read the gps data files are in this class.
  if not 'HyperSpec' in dir():
    print("Can't find the HyperSpec class.  You need to run the Cell above: 'Load all local defs.'")
  else:
    hs = HyperSpec()
    gps_df = hs.read_gps_file_to_df(Gps_File_Name)
    screenProj = Proj(init='epsg:3857')     # 
    wgs84Proj = Proj(init='epsg:4326')    # WGS-84

    map_border = .1
    lat_min = gps_df['Lat'].min() - map_border
    lat_max = gps_df['Lat'].max() + map_border
    lon_min = gps_df['Lon'].min() - map_border
    lon_max = gps_df['Lon'].max() + map_border
    # print( lat_max, lat_min, lon_max,lon_min)

    world_lon1, world_lat1 = transform(wgs84Proj,screenProj,lon_min,lat_max)
    world_lon2, world_lat2 = transform(wgs84Proj,screenProj,lon_max,lat_min)

    if 'gps_df' not in globals():
      print('\nNo gps_df dataframe is present.  You need to process some data to create a "gps_df" variable.')
    else:
      gps_lats, gps_lons = transform(wgs84Proj, screenProj, gps_df['Lon'], gps_df['Lat'])

      #cartodb = get_provider(CARTODBPOSITRON)
      if mp == "ESRI Imagery":
        cartodb = get_provider(ESRI_IMAGERY)
      elif mp == "WikiMedia":
        cartodb = get_provider(WIKIMEDIA)
      elif mp == "WorldMap":
        cartodb = get_provider(CARTODBPOSITRON)
      elif mp == "Open Street Maps":
        cartodb = get_provider(OSM)
      elif mp == "STAMEN_TERRAIN":
        cartodb = STAMEN_TERRAIN

      fig = figure(title=Gps_File_Name,
                  plot_width=800, plot_height=800,
                  tooltips=TOOLTIPS, tools=TOOLS,
                  x_range=(world_lon1, world_lon2),
                  y_range=(world_lat1, world_lat2),
                  x_axis_type="mercator", y_axis_type="mercator",
                  output_backend="webgl"
                  )


      fig.add_tile(cartodb)
      fig.circle( y=gps_lons, x=gps_lats, size=Point_Size, color=Track_Color )
      show(fig)



In [ ]:
#@title Show the GPS Track on a map with data results. { form-width: "30%" }
if __name__ == '__main__':
  Gps_File_Name = "/content/drive/MyDrive/Missions/2021-1208-HAB-Lake-Parker/2021-1208/all-gps.txt" #@param {type:"string"}
  mp = "ESRI Imagery" #@param ["ESRI Imagery", "Open Street Maps", "WorldMap" ]
  Track_Color = "red" #@param ["red", "green", "blue", "black", "white", "gray", "lightgray", "darkgray"]
  Point_Size = .1 #@param [".1", ".25", ".5", ".75", "1.0", "1.5", "2.0", "3.0", "5.0", "10", "20"] {type:"raw"}
  Fl_Scale =  .3#@param {type:"number"}
  Show_Index = True #@param ["True", "False"] {type:"raw"}


  import pandas as pd
  from pathlib import Path

  if not Path('/usr/local/bin/pyproj').is_file():
    !pip install pyproj
  import bokeh.io
  bokeh.io.output_notebook()
  import bokeh.plotting
  from bokeh.plotting import figure
  from bokeh.io import output_notebook, show
  from bokeh.tile_providers import get_provider, WIKIMEDIA, CARTODBPOSITRON, STAMEN_TERRAIN, STAMEN_TONER, ESRI_IMAGERY, OSM
  from pyproj import Proj, transform

  import warnings
  warnings.filterwarnings("ignore")

  TOOLS = "pan,wheel_zoom,box_zoom,reset, save, undo, redo, hover"
  TOOLTIPS = [
      ("index", "$index")  #,
      #("(x,y)", "($x, $y)"),
      #("desc", "@desc"),
  ]

  # Create a spectra obj
  if not 'HyperSpec' in dir():
    print("Can't find the HyperSpec class.  You need to run the Cell above: 'Load all local defs.'")
  else:
    hs = HyperSpec()
    gps_df = hs.read_gps_file_to_df(Gps_File_Name)


  screenProj = Proj(init='epsg:3857')     # 
  wgs84Proj = Proj(init='epsg:4326')    # WGS-84

  map_border = .1
  lat_min = gps_df['Lat'].min() - map_border
  lat_max = gps_df['Lat'].max() + map_border
  lon_min = gps_df['Lon'].min() - map_border
  lon_max = gps_df['Lon'].max() + map_border
  # print( lat_max, lat_min, lon_max,lon_min)

  world_lon1, world_lat1 = transform(wgs84Proj,screenProj,lon_min,lat_max)
  world_lon2, world_lat2 = transform(wgs84Proj,screenProj,lon_max,lat_min)

  #gps_lats, gps_lons = transform(wgs84Proj, screenProj, gps_df['Lon'], gps_df['Lat'])
  if 'spec_df' not in globals():
    print('\nNo spec_df dataframe is present.  You need to process some data to create a "spec_df" variable.')
  else:
    gps_lats, gps_lons = transform(wgs84Proj, screenProj, spec_df['Lon'], spec_df['Lat'])

    #cartodb = get_provider(CARTODBPOSITRON)
    if mp == "ESRI Imagery":
      cartodb = get_provider(ESRI_IMAGERY)
    elif mp == "WikiMedia":
      cartodb = get_provider(WIKIMEDIA)
    elif mp == "WorldMap":
      cartodb = get_provider(CARTODBPOSITRON)
    elif mp == "Open Street Maps":
      cartodb = get_provider(OSM)
    elif mp == "STAMEN_TERRAIN":
      cartodb = STAMEN_TERRAIN

    fig = figure(title=Gps_File_Name,
                plot_width=800, plot_height=800,
                tooltips=TOOLTIPS, tools=TOOLS,
                x_range=(world_lon1, world_lon2),
                y_range=(world_lat1, world_lat2),
                x_axis_type="mercator", y_axis_type="mercator",
                output_backend="webgl"
                )
    from bokeh.models import Title
    fig.add_layout( Title(text=f'Data Source: {Root_Data_Path}', align="center"), "below")

    fig.add_tile(cartodb)
  
    min_chl = spec_df['Chl_Fl'].min() + 0.001
    min_700 = spec_df['fl_700'].min() + 0.001
    if Show_Index:
      fig.circle(y=gps_lons, x=gps_lats, legend_label="", color='Yellow', size=4 )
    fig.ray(y=gps_lons, x=gps_lats, length=(spec_df['fl_700']-min_700)*Fl_Scale, angle_units='deg', angle=-spec_df['Course'], 
            line_color='Orange', line_width=3, legend_label="700nm")
    fig.ray(y=gps_lons, x=gps_lats, length=(spec_df['Chl_Fl']-min_chl)*Fl_Scale, angle_units='deg', angle=-spec_df['Course']+180.0,
            line_color='Red', line_width=3, legend_label="683nm")

    show(fig)



## Plot a single json spectra file v2.
See [Fraunhofer lines](https://en.wikipedia.org/wiki/Fraunhofer_lines) for absorption lines that can be used for wavelength calibration.

Also See: [Strong Lines of Mercury ( Hg )](https://physics.nist.gov/PhysRefData/Handbook/Tables/mercurytable2.htm)

### Plot Settings.

In [ ]:
#@title Plot Spectra Settings { form-width: "30%", display-mode: "form"}

if __name__ == '__main__':
  Plot_Spectra_Settings = True    # used to indicate this cell has been run.
  Spectra_Plot_Style = "Lines" #@param ["Lines", "Lines & Points", "Points"]
  Point_Size = 5.0 #@param [".1", ".25", ".5", ".75", "1.0", "1.5", "2.0", "3.0", "5.0"] {type:"raw"}
  Line_Color  = "blue" #@param ["red", "green", "blue", "black", "white", "gray", "lightgray", "darkgray"]
  Point_Color = "green" #@param ["red", "green", "blue", "black", "white", "gray", "lightgray", "darkgray"]
  X_Axis = "Wavelength (nm)" #@param ["Wavelength (nm)", "Pixels"]
  Remove_Bias = True #@param ["True", "False"] {type:"raw"}
  Y_Axis = "Average" #@param ["Average", "Sum Total"]
  Show_Calibration_Lines = "Fraunhofer Lines" #@param ["Fluorescent Light Lines", "Fraunhofer Lines", "None"]
  Show_RGB = True #@param ["True", "False"] {type:"raw"}

  print('Spectral Plot Setting:\n'
  f'    Spectra_Plot_Style: {Spectra_Plot_Style}\n'
  f'            Point_Size: {Point_Size}               Line_Color:{Line_Color}   Point_Color:{Point_Color}\n'
  f'                X_Axis: {X_Axis}  Remove_Bias:{Remove_Bias}        Y_Axis:{Y_Axis}\n'
  f'Show_Calibration_Lines: {Show_Calibration_Lines}    Show_RGB:{Show_RGB}\n')


### Plot a single Spectra and nearby Photo

In [ ]:
#@title Plot a Spectra File using above settings{ form-width: "30%" }
if __name__ == '__main__':
  Spectra_File_Name = "/content/drive/MyDrive/Missions/2021-1208-HAB-Lake-Parker/2021-1208/183807/hab_spectra/2021-1208-184034-847363-spec.json" #@param {type:"string"}
  Index_Or_File = "File" #@param ["Index", "File"]
  File_Index =  550#@param {type:"integer"}

  import os
  import pandas as pd
  import numpy as np
  import bokeh.io
  from bokeh.plotting import figure, show
  from bokeh.io import output_notebook, show
  from bokeh.layouts import row

  bokeh.io.output_notebook()

  if Index_Or_File == 'Index':
    Spectra_File_Name = spec_df['Json_spec'].iloc[File_Index]
  else:
    pass


  # Create a spectra obj
  if not 'HyperSpec' in dir():
    print("Can't find the HyperSpec class.  You need to run the Cell above: 'Load all local defs.'")
  else:
    if not 'Plot_Spectra_Settings' in globals():
      print("You must first run the cell: 'Plot Spectra Settings'  to load the plot settings.  ")
    else:
      hs = HyperSpec()
      # Use a json spectra to initialize things.
      hs.configure_json_spectra(Spectra_File_Name)

      # Calibrate the spectra for wavelength in nanometers
      hs.calibrate_using_2_wavelengths(pixel0=73, wavelength0=430.774, pixel1=941, wavelength1=759.370 )

      # Read a spectra array from a file into s
      y = hs.read_spectra(Spectra_File_Name, 
                          remove_bias = Remove_Bias,
                          y_average = True if Y_Axis == 'Average' else False
                          )
      
      y_max = y.max()

      # Setup X-Axis as pixel location or wavelength
      if X_Axis == "Wavelength (nm)":
        x = hs.wavelengths
      else:
        x = hs.Xpixels

      TOOLS = "pan,wheel_zoom,box_zoom,reset, save, undo, redo, hover"
      TOOLTIPS = [
          ("index", "$index"),
          ("(x,y)", "($x, $y)"),
          ("desc", "@desc"),
      ]

      ttl = f'Spectral Flie:  {os.path.split(Spectra_File_Name)[1]}'
      spec_fig = figure(title=ttl, 
                        x_axis_label=X_Axis, 
                        tooltips=TOOLTIPS, tools=TOOLS, 
                        y_axis_label='Relative Digital Counts', 
                        width=1000, height=400,
                        output_backend="webgl"
                        )

      spec_fig.title.text_font_size='24px'
      spec_fig.xaxis.axis_label_text_font_size = '24pt'
      spec_fig.xaxis.ticker.num_minor_ticks = 5

      spec_fig.xaxis.ticker.desired_num_ticks = 10
      spec_fig.xaxis.major_label_text_font_size = "18pt"

      #hover_tool = HoverTool(tooltips=[
      #            ('Value', '$y'),
      #            ('Date', '@date_readable') ], renderers=[line] )
      #self.plot.tools.append(hover_tool)

      if Show_Calibration_Lines == "Fraunhofer Lines":
        cal_src = hs.Fraunhofer_lines
        cal_lbl = "Fraunhofer Lines"
      elif Show_Calibration_Lines == "Fluorescent Light Lines":
        cal_src = hs.HG_lines
        cal_lbl = "Fluorescent Light Lines"

      if Show_Calibration_Lines != 'None':
        for frl in cal_src:
          xc = cal_src[frl][1]
          spec_fig.line([xc,xc],[0,y_max], legend_label=cal_lbl, line_width=2, color='LightGray')

      if Spectra_Plot_Style == "Lines" or Spectra_Plot_Style == "Lines & Points":
        spec_fig.line(x,y, legend_label="Down Looking Spectra", line_width=2, color=Line_Color)
      if Spectra_Plot_Style == "Points" or Spectra_Plot_Style == "Lines & Points":
        spec_fig.circle(x,y, size=Point_Size, hover_color='white', hover_alpha=0.5, color=Point_Color)

      show(spec_fig)
      # Show nearby RGB image.
      if Show_RGB == True:
        try:
          show_RGB_at_spec( Spectra_File_Name )
        except:
          print('RGB Photo not found.')



## Plot Animated spectra and photos along a selected flightline.

### Animate Spectra from a flightline

In [ ]:
#@title Animate a flight line of spectra { form-width: "30%" }

if __name__ == '__main__':
  from IPython.display import clear_output
  import os
  #from   matplotlib import rc
  from   matplotlib import pyplot as plt
  import matplotlib as mpl
  import numpy as np
  import pandas as pd
  import warnings
  warnings.filterwarnings("ignore")

  from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

  # /content/drive/MyDrive/Missions/2021-0717-HAB-pie/165347   Lots of fl.

  #Spectra_File_Name = "/content/drive/MyDrive/Missions/2021-0717-HAB-pie/161618/hab_spectra/2021-0717-161618-720442-spec.json" #@param {type:"string"}
  Flight_Line_Dir = "/content/drive/MyDrive/Missions/2021-1208-HAB-Lake-Parker/2021-1208/190829" #@param {type:"string"}
  X_Axis = "Wavelength (nm)" #@param ["Wavelength (nm)", "Pixels"]
  Plot_Fill = "Area Fill" #@param ["Area Fill", "Line Only"]


  fp = Flight_Line_Dir+"/hab_spectra/"
  l = os.listdir(fp)
  spec = pd.read_json(path_or_buf =fp+ l[0])

  #spec = pd.read_json(path_or_buf =Spectra_File_Name)
  y = np.array(spec['hab_spec'].spectra) / 800.0
  y = y - y.min()

  # Setup X-Axis as pixel location or wavelength
  x_pix = np.arange(0,y.size ) 
  w = 400 + x_pix * ((765-400)/944)

  if X_Axis == "Wavelength (nm)":
    x = w
  else:
    x = x_pix

  cc = 'blue'
  for xx in l:
      sfn = fp+xx
      # Show nearby RGB image.
      if Show_RGB == True:
        try:
          show_RGB_at_spec( sfn )
        except:
          pass
      spec = pd.read_json(path_or_buf = sfn )

      y = np.array(spec['hab_spec'].spectra) / 800.0
      y = y - y.min()
      #plt.cla()
      

      fig = plt.figure(figsize=(15,6))
      ax = plt.axes()
      # 2021-0717-161618-720442-spec.json
      ttl = xx.split(sep='-')
      water = hs.is_water(x,y)
      mpl.pyplot.title( f'Date:{ttl[1]}, {ttl[0]}  HMS:{ttl[-3]} utc  {water:4.4f} ', fontsize=20)
      mpl.pyplot.ylim(0,256)
      if X_Axis == "Wavelength (nm)":
        mpl.pyplot.xlim(400,900)
        ax.xaxis.set_major_locator(MultipleLocator(20))
        plt.xlabel('Wavelength (nm)', fontsize=18)
        plt.ylabel('Digital Counts', fontsize=18)
      else:
        mpl.pyplot.xlim(0,1280)
        ax.xaxis.set_major_locator(MultipleLocator(50))
        plt.xlabel('Sensor Pixel Number', fontsize=18)
        plt.ylabel('Digital Counts', fontsize=18)
      ax.xaxis.grid() # vertical lines
      plt.plot(x, y, color='green', linewidth=3, label='Down Looking' )
      if Plot_Fill == "Area Fill":
        if water < 4.0:
          cc = "blue"
        else:
          cc = 'Brown'
        plt.fill_between(x,y, alpha=.25, color=cc)
      plt.legend()
      plt.tight_layout()
      clear_output(wait=True)
      plt.pause(.2)
  print('Completed.')



### Plot Signals vs time

In [ ]:
#@title Plot CHL-Fl vs Seconds Of Day { form-width: "30%" }
if __name__ == '__main__':
  from bokeh.plotting import figure, show
  from bokeh.io import output_notebook, show
  import bokeh
  if __name__ == '__main__':
    bokeh.io.output_notebook()
    p = figure(title="Chl Fl", x_axis_label="Seconds of the day", y_axis_label="Chl Fl", width=1000)
    p.line(spec_df['sod'], spec_df['Chl_Fl']-10, legend_label="CHL Fl.", color='Red')
    p.line(spec_df['sod'], spec_df['fl_700'], legend_label="Fl 700.", color='Blue')
    show(p)


### Plot CHL_Fl as filled circles vs lat/lon

In [ ]:
#@title Plot CHL-Fl vs Lat / Lon { form-width: "30%" }
if __name__ == '__main__':
  from bokeh.plotting import figure, show
  from bokeh.io import output_notebook, show
  import bokeh
  if __name__ == '__main__':
    bokeh.io.output_notebook()
    p = figure(title="Chl Fl", x_axis_label="Seconds of the day", y_axis_label="Chl Fl", width=1000)
    p.circle(spec_df['Lon'], spec_df['Lat'], legend_label="CHL Fl.", color='Red', size=(spec_df['fl_700'])/6)
    #p.circle(df['sod'], df['fl_700'], legend_label="Fl 700.", color='Blue')
    show(p)


# Code under Development

In [ ]:
! cat /content/drive/MyDrive/Missions/2021-1208-HAB-Lake-Parker/2021-1208/183807/2021-1208-183605-002334-flightline-gps.txt /content/drive/MyDrive/Missions/2021-1208-HAB-Lake-Parker/2021-1208/190829/2021-1208-190428-678619-flightline-gps.txt >/content/drive/MyDrive/Missions/2021-1208-HAB-Lake-Parker/2021-1208/all-gps.txt

## Iterate over all json, and image files

In [ ]:
#@title Iterate over all json, and image files { form-width: "30%" }
if __name__ == '__main__':
  Root_Path = "/content/2021-0717-HAB-pie-NO-PIX" #@param {type:"string"}
  variable_name = "" #@param {type:"string"}


  from IPython.display import clear_output
  import os
  #from   matplotlib import rc
  #from   matplotlib import pyplot as plt
  #import matplotlib as mpl
  import numpy as np
  import pandas as pd
  from pathlib import Path
  import warnings
  warnings.filterwarnings("ignore")


## Work with files and dirs

In [ ]:
#@title Add a Colabs Title { form-width: "30%" }
if __name__ == '__main__':
  # From the mission directory, get a complete list of the flightline subdirectories.
  print('\nTesting hs.get_list_of_flight_lines(\'/content/drive/MyDrive/Missions/2021-0717-HAB-pie\')')
  flight_line_list = hs.get_list_of_flight_lines('/content/drive/MyDrive/Missions/2021-0717-HAB-pie')
  print(f'\n{len(flight_line_list)} Total flightlines found.  flight_line_list[1:5]:')
  for f in flight_line_list[1:5]:
    print( f'  {f}' )

  js = hs.get_list_of_hab_files(flight_line_list[1], subdir='hab_spectra', ext='json')

  spec_images = hs.get_list_of_hab_files(flight_line_list[1], subdir='hab_images', ext='jpg')
  print('')
  print(f'    Spec_Images: {len(spec_images):5d}')

  print(f'Json Spec Files: {len(js):5d}' )

  photos = hs.get_list_of_hab_files(flight_line_list[1], subdir='hab_rgb', ext='jpg')
  print(f'         Photos: {len(photos):5d}')

  flight_lines = hs.get_list_of_flight_lines('/content/drive/MyDrive/Missions/2021-0717-HAB-pie')
  print(f'    FlightLines: {len(flight_lines):5d}   {flight_lines[0:2]}'  )

  i = 0
  t = len(flight_lines)
  all_specs = []
  for fl in flight_lines:
    # Process a single flightline
    print(f'  Flight_line {i} of {t}: {fl}')
    specs = hs.get_list_of_hab_files(fl, subdir='hab_spectra', ext='json')
    all_specs.extend(specs)
    i += 1
    for s in specs[0:5]:
      #print(f'  {s}')
      pass
  print(f'{hs.now_utc()}: Operation Completed.')



# Generate HabSpec Class library


In [ ]:
#@title Clone HabSpec from GitHub and convert to python. { form-width: "30%" }
# Test if running in main.
if __name__ == '__main__':
  ! git clone https://github.com/lidar532/RedTideProcessing.git
  ! cd /content/RedTideProcessing/; jupyter nbconvert --to python hab_process.ipynb
  print(f'{hs.now_utc()}: Operation Completed.')


In [ ]:
#@title Copy HabSpec from Gdrive to /content/RedTideProcessing { form-width: "30%" }
# Test if running in main.
if __name__ == '__main__':
  import datetime
  ! mkdir -p RedTideProcessing
  ! cp -v /content/drive/MyDrive/Colab\ Notebooks/hab_process.ipynb /content/RedTideProcessing/
  print(f'{datetime.datetime.utcnow()}: Operation Completed.')

In [ ]:
#@title Convert the HabSpec notebook to python.py { form-width: "30%" }
# Test if running in main.
if __name__ == '__main__':
  ! cd /content/RedTideProcessing/; jupyter nbconvert --to python hab_process.ipynb
  print(f'{datetime.datetime.utcnow()}: Conversion Completed.')

In [ ]:
if __name__ == '__main__':
  !rm -rf /content/RedTideProcessing/

In [ ]:
if __name__ == '__main__':
  #from RedTideProcessing.hab_process import HyperSpec
  import RedTideProcessing.hab_process



In [ ]:
if __name__ == '__main__':
  hhh = RedTideProcessing.hab_process.HyperSpec()
  print(hhh.asof_str, hhh.create_time)

In [ ]:
if __name__ == '__main__':
  RedTideProcessing.hab_process.HyperSpec.asof_str

In [ ]:
if __name__ == '__main__':
  reload(RedTideProcessing.hab_process)

In [ ]:
#@title reload(lib):  Reloads a library after making changes.
if __name__ == '__main__':
  from importlib import reload   # get the reload functions.
  import matplotlib as mp        # Load a library
  reload(mp)                     # Reload the library.

# References

[Novel Optical Techniques for Detecting and Classifying Toxic Dinoflagellate Karenia Brevis Blooms Using Satellite Imagery](https://www.researchgate.net/figure/Modeled-remote-sensing-reflectance-spectra-for-K-brevis-cell-concentrations-a-greater_fig1_26237728)

-------
## Links

[Readme.md for RedTideProcessing](https://github.com/lidar532/RedTideProcessing/blob/main/README.md)

[Discussion ](https://github.com/lidar532/RedTideProcessing/discussions)

[Issues Tracker](https://github.com/lidar532/RedTideProcessing/issues)

[RedTideProcessing Wiki](https://github.com/lidar532/RedTideProcessing/wiki)
